In [1]:
import pandas as pd
import numpy as np
import os
from itertools import product
from tqdm.notebook import tqdm
import calendar

In [2]:
DATA_FOLDER = '../Data/'

sales           = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train.csv.gz'))
items           = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv'))
# item_categories = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories.csv'))
# shops           = pd.read_csv(os.path.join(DATA_FOLDER, 'shops.csv'))
test            = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv.gz'))
# samplesub       = pd.read_csv(os.path.join(DATA_FOLDER, 'sample_submission.csv.gz'))

In [3]:
sales['item_category_id'] = sales.item_id.map(items.item_category_id)
sales.item_cnt_day = sales.item_cnt_day.clip(0, 20)
sales = sales[sales.item_price < 30000]

In [4]:
# index_cols = ['date_block_num', 'shop_id', 'item_id']

# train = []
# block = sales.date_block_num.unique()

# for shop in tqdm(sales.shop_id.unique()):
#     cur_items = sales.loc[sales.shop_id == shop, 'item_id'].unique()
#     train.append(
#         np.array(list(
#             product(*[block, [shop], cur_items])
#         ))
#     )
    
# train = pd.DataFrame(np.vstack(train), columns=index_cols, dtype=np.int32)

In [5]:
index_cols = ['date_block_num', 'shop_id', 'item_id']

train = []
# block = sales.date_block_num.unique()

for shop in tqdm(sales.shop_id.unique()):
    cur_items = sales.loc[sales.shop_id == shop, 'item_id'].unique()
    cur_block = sales.loc[sales.shop_id == shop, 'date_block_num'].unique()
    train.append(
        np.array(list(
            product(*[cur_block, [shop], cur_items])
        ))
    )
    
train = pd.DataFrame(np.vstack(train), columns=index_cols, dtype=np.int32)

In [7]:
index_cols = ['date_block_num', 'shop_id', 'item_id']

train = []
# block = sales.date_block_num.unique()

for item in tqdm(sales.item_id.unique()):
    cur_shop = sales.loc[sales.item_id == item, 'shop_id'].unique()
    cur_block = sales.loc[sales.item_id == item, 'date_block_num'].unique()
    train.append(
        np.array(list(
            product(*[cur_block, cur_shop, [item]])
        ))
    )
    
train = pd.DataFrame(np.vstack(train), columns=index_cols, dtype=np.int32)

In [8]:
train.shape

(6804497, 3)

In [36]:
test.head()

,ID,shop_id,item_id,date_block_num
0,0,5,5037,34
1,1,5,5320,34
2,2,5,5233,34
3,3,5,5232,34
4,4,5,5268,34


In [35]:
test['date_block_num'] = np.full(test.shape[0], 34)

In [29]:
train.tail()

,date_block_num,shop_id,item_id,item_cnt_month,year,month,num_days,num_mon,num_tue,num_wed,...,price,price_month,price_shop,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,item_cnt_month_lag_12,item_cnt_month_lag_24,avg_cnt_month,avg_cnt_shop
14417423,33,36,20231,1.0,2015,10,31,4,4,4,...,129.375,972.5,1166.0,0.0,0.0,0.0,0.0,0.0,0.733398,0.430176
14417424,33,36,20334,1.0,2015,10,31,4,4,4,...,1949.000,972.5,1166.0,0.0,0.0,0.0,0.0,0.0,0.500000,0.430176
14417425,33,36,20339,1.0,2015,10,31,4,4,4,...,1418.000,972.5,1166.0,0.0,0.0,0.0,0.0,0.0,0.466553,0.430176
14417426,33,36,20352,1.0,2015,10,31,4,4,4,...,1549.000,972.5,1166.0,0.0,0.0,0.0,0.0,0.0,0.611328,0.430176
14417427,33,36,20602,1.0,2015,10,31,4,4,4,...,1549.000,972.5,1166.0,0.0,0.0,0.0,0.0,0.0,0.833496,0.430176


In [6]:
group = sales.groupby(index_cols).agg({'item_cnt_day': 'sum'})
group.columns = ['item_cnt_month']
group.reset_index(inplace= True)
train = pd.merge(train, group, on= index_cols, how= 'left')
train.item_cnt_month = train.item_cnt_month.fillna(0).clip(0, 20)

In [7]:
def num_days(block):
    y = int(2013 + block/12)
    m = 1 + block % 12
    return max(calendar.monthcalendar(y, m)[-1])

def wday(block, d):
    y = int(2013 + block/12)
    m = 1 + block % 12
    cal = calendar.monthcalendar(y, m)
    return sum([1 for w in cal if w[d]])

In [8]:
train.head()

,date_block_num,shop_id,item_id,item_cnt_month
0,0,59,22154,1.0
1,0,59,22151,2.0
2,0,59,5603,1.0
3,0,59,5587,2.0
4,0,59,5613,1.0


In [9]:
train['year'] = (train.date_block_num/12 + 2013).astype(int)
train['month'] = (train.date_block_num % 12 + 1).astype(int)

In [11]:
train['num_days'] = train.date_block_num.map(pd.Series([num_days(i) for i in range(40)]))

for d, wk in enumerate(['mon', 'tue', 'wed', 'thur', 'fri', 'sat', 'sun']):
    train['num_'+ wk] = train.date_block_num.map(pd.Series([wday(i, d) for i in range(40)]))

In [12]:
prices       = sales.groupby('item_id' ).item_price.mean()
prices_month = sales.groupby('month'   ).item_price.mean()
prices_shop  = sales.groupby('shop_id' ).item_price.mean()

In [13]:
train['item_category_id'] = train.item_id.map(items.item_category_id)
train['price'] = train.item_id.map(prices).astype('float16')
train['price_month'] = train.month.map(prices_month).astype('float16')
train['price_shop'] = train.shop_id.map(prices_shop).astype('float16')

In [14]:
train.head()

,date_block_num,shop_id,item_id,item_cnt_month,year,month,num_days,num_mon,num_tue,num_wed,num_thur,num_fri,num_sat,num_sun,item_category_id,price,price_month,price_shop
0,0,59,22154,1.0,2013,1,31,4,5,5,5,4,4,4,37,703.000,883.5,883.5
1,0,59,22151,2.0,2013,1,31,4,5,5,5,4,4,4,40,380.000,883.5,883.5
2,0,59,5603,1.0,2013,1,31,4,5,5,5,4,4,4,5,462.750,883.5,883.5
3,0,59,5587,2.0,2013,1,31,4,5,5,5,4,4,4,5,193.625,883.5,883.5
4,0,59,5613,1.0,2013,1,31,4,5,5,5,4,4,4,2,6052.000,883.5,883.5


In [15]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [16]:
train.head()

,date_block_num,shop_id,item_id,item_cnt_month,year,month,num_days,num_mon,num_tue,num_wed,num_thur,num_fri,num_sat,num_sun,item_category_id,price,price_month,price_shop
0,0,59,22154,1.0,2013,1,31,4,5,5,5,4,4,4,37,703.000,883.5,883.5
1,0,59,22151,2.0,2013,1,31,4,5,5,5,4,4,4,40,380.000,883.5,883.5
2,0,59,5603,1.0,2013,1,31,4,5,5,5,4,4,4,5,462.750,883.5,883.5
3,0,59,5587,2.0,2013,1,31,4,5,5,5,4,4,4,5,193.625,883.5,883.5
4,0,59,5613,1.0,2013,1,31,4,5,5,5,4,4,4,2,6052.000,883.5,883.5


In [17]:
def lag_feature(df, lags, col='item_cnt_month'):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in tqdm(lags):
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id', col+'_lag_'+str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
        df[col+'_lag_'+str(i)] = df[col+'_lag_'+str(i)].astype('float16')
    return df

In [18]:
train = lag_feature(train, [1, 2, 3, 12, 24])

In [23]:
def add(train, index_cols, name, col='item_cnt_month', func='mean'):
    group = train[train.date_block_num.isin(sales.date_block_num.unique())].groupby(index_cols).agg({col: func})
    group.columns = [name]
    group.reset_index(inplace= True)
    train = pd.merge(train, group, on= index_cols, how= 'left')
    train[name] = train[name].fillna(0).astype('float16')
    return train
    return lag_feature(train, [1, 2, 3], name)

In [24]:
test.item_id.isin(train.item_id.unique()).all()

False

In [25]:
test.shop_id.isin(train.shop_id.unique()).all()

True

In [26]:
train = add(train, ['month', 'item_id'], 'avg_cnt_month')
train = add(train, ['month', 'shop_id'], 'avg_cnt_shop')

In [27]:
train.to_hdf('../Dump/train.h5', 'train', 'w')

In [28]:
train.head()

,date_block_num,shop_id,item_id,item_cnt_month,year,month,num_days,num_mon,num_tue,num_wed,...,price,price_month,price_shop,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,item_cnt_month_lag_12,item_cnt_month_lag_24,avg_cnt_month,avg_cnt_shop
0,0,59,22154,1.0,2013,1,31,4,5,5,...,703.000,883.5,883.5,NaN,NaN,NaN,NaN,NaN,0.230713,0.213745
1,0,59,22151,2.0,2013,1,31,4,5,5,...,380.000,883.5,883.5,NaN,NaN,NaN,NaN,NaN,1.810547,0.213745
2,0,59,5603,1.0,2013,1,31,4,5,5,...,462.750,883.5,883.5,NaN,NaN,NaN,NaN,NaN,0.076904,0.213745
3,0,59,5587,2.0,2013,1,31,4,5,5,...,193.625,883.5,883.5,NaN,NaN,NaN,NaN,NaN,0.503418,0.213745
4,0,59,5613,1.0,2013,1,31,4,5,5,...,6052.000,883.5,883.5,NaN,NaN,NaN,NaN,NaN,0.386719,0.213745
